In [11]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
from glob import glob
import ast
from time import sleep
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
data_folder = ''
api = ''

## Drugs

In [4]:
ingredient_df = pd.read_csv(data_folder+'data/final_UK/ingredient_data.csv')
ingredient_df['ingredient_id'] = ingredient_df.ingredient_id.apply(lambda x: str(x.split('/')[-1]))
ingredient_dict = dict(zip(ingredient_df.ingredient_name, ingredient_df.ingredient_id))
ingredient_rxnorm_dict = dict(zip(ingredient_df.ingredient_id, ingredient_df.concept_code))
ingredient_df.head()

,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
0,1524,"2,4-dichlorobenzyl alcohol",NaN,22906.0,dichlorobenzyl alcohol
1,3087,5-aminolevulinic acid,NaN,683.0,aminolevulinic acid
2,2642,5-aminolevulinic acid hydrochloride,NaN,261715.0,aminolevulinic acid hydrochloride
3,1388,abacavir,1736971.0,190521.0,abacavir
4,1141,abacavir sulfate,19052381.0,221052.0,abacavir sulfate


In [5]:
drug_df = pd.read_csv(data_folder+'data/final_UK/drug_data.csv')
drug_df['ingredient_ids'] = drug_df.active_ingredients.apply(lambda x: [ingredient_dict[i] for i in x.split(', ') if i in ingredient_dict.keys()])
drug_df['ingredient_rxnorm'] = drug_df.ingredient_ids.apply(lambda x: sorted([ingredient_rxnorm_dict[i] for i in x]))
drug_df['product_id'] = drug_df.product_id.apply(lambda x: x.split('/')[-2])
drug_df['product_name'] = drug_df.product_name.apply(lambda x: x.strip())
drug_df = drug_df[['ingredient_rxnorm', 'active_ingredients', 'product_id', 'product_name']].drop_duplicates(subset=['product_id', 'product_name'])
drug_df['ingredient_rxnorm'] = drug_df.ingredient_rxnorm.apply(lambda x: ', '.join([str(int(i)) for i in x if str(i) != 'nan']))
drug_df.head(1)

,ingredient_rxnorm,active_ingredients,product_id,product_name
0,"508, 22906, 288088","allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",1589,Boots Antiseptic Cream


In [6]:
drugs = drug_df[drug_df.ingredient_rxnorm != '']
print(drugs.shape, len(drugs.ingredient_rxnorm.unique().tolist()))

(8848, 4) 1835


In [9]:
drug_df.to_csv(data_folder+'data/final_UK/ingredients.csv', index=False)

## Conditions

In [16]:
#free-text data
free_text_df = pd.read_csv(data_folder+'data/ade_text_table_onsides_pred_v0924.csv')
free_text_df = free_text_df[free_text_df.pt_meddra_id.notna()][['label_id', 'pt_meddra_id', 'pt_meddra_term']]
free_text_df['pt_meddra_id'] = free_text_df['pt_meddra_id'].apply(lambda x: int(float(x)))
free_text_df['pt_meddra_term'] = free_text_df['pt_meddra_term'].apply(lambda x: x.lower())
free_text_df = free_text_df.drop_duplicates()
free_text_df.head(1)

,label_id,pt_meddra_id,pt_meddra_term
0,1589,10020751.0,Hypersensitivity


In [26]:
free_text_df.head()

,label_id,pt_meddra_id,pt_meddra_term
0,1589,10020751,hypersensitivity
2,1589,10037211,psychomotor hyperactivity
3,4223,10002424,angioedema
4,4223,10030095,oedema
6,4223,10020751,hypersensitivity


In [35]:
conditions = pd.read_csv(data_folder+'data/drug_ade_data_parsed.csv')
conditions['product_id'] = conditions['product_id'].apply(lambda x: str(x))
conditions['matched_codes'] = conditions['matched_codes'].apply(lambda x: ast.literal_eval(x) if str(x) != 'nan' else None)
conditions['exact_match_list'] = conditions['exact_match_list'].apply(lambda x: ast.literal_eval(x) if str(x) != 'nan' else None)
conditions = conditions[['product_id', 'freq', 'exact_match_list', 'matched_codes']].explode(['exact_match_list', 'matched_codes'])
conditions['freq'] = conditions.freq.apply(lambda x: 'not known' if str(x) == 'nan' else x)
conditions.columns = ['product_id', 'freq', 'pt_meddra_term', 'pt_meddra_id']
conditions = conditions[conditions.pt_meddra_term.notna()]
conditions = conditions.drop_duplicates(['pt_meddra_term', 'pt_meddra_id', 'product_id'])
conditions.head()

,product_id,freq,pt_meddra_term,pt_meddra_id
0,4223,not known,hypersensitivity,10020751
1,4223,not known,glossodynia,10018388
1,4223,not known,oral discomfort,10030973
1,4223,not known,discomfort,10013082
2,5606,not known,hypersensitivity,10020751


In [36]:
free_text_df['freq'] = 'not known'
free_text_df.columns = ['product_id', 'pt_meddra_id', 'pt_meddra_term', 'freq']
cond = pd.concat([conditions, free_text_df])
print(cond.shape[0])
cond['product_id'] = cond['product_id'].astype(str)
cond = cond.drop_duplicates(['pt_meddra_term', 'pt_meddra_id', 'product_id'])
print(cond.shape[0])
cond.head()

684192
639297


,product_id,freq,pt_meddra_term,pt_meddra_id
0,4223,not known,hypersensitivity,10020751
1,4223,not known,glossodynia,10018388
1,4223,not known,oral discomfort,10030973
1,4223,not known,discomfort,10013082
2,5606,not known,hypersensitivity,10020751


## Make raw table

In [54]:
drug_df = pd.read_csv(data_folder+'data/final_UK/ingredients.csv')
drug_df['product_id'] = drug_df['product_id'].astype(str)
drug_df.head(1)

,ingredient_rxnorm,active_ingredients,product_id,product_name
0,"508, 22906, 288088","allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",1589,Boots Antiseptic Cream


In [56]:
raw_tally = cond.merge(drug_df[['ingredient_rxnorm', 'active_ingredients', 'product_id']], on = 'product_id', how = 'left')
#raw_tally = raw_tally[['product_id', 'ingredient_rxnorm', 'active_ingredients', 'pt_meddra_id', 'pt_meddra_term', 'freq']]
raw_tally.to_csv(data_folder+'data/final_UK/adverse_events_all_labels_w_freq.csv', index=False)
raw_tally.head(1)

,product_id,freq,pt_meddra_term,pt_meddra_id,ingredient_rxnorm,active_ingredients
0,4223,not known,hypersensitivity,10020751,"22906, 236248","amylmetacresol, 2,4-dichlorobenzyl alcohol"


## Combine Data

In [47]:
drug_df = pd.read_csv(data_folder+'data/final_UK/ingredients.csv')
drug_df.head(1)

,ingredient_rxnorm,active_ingredients,product_id,product_name
0,"508, 22906, 288088","allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",1589,Boots Antiseptic Cream


In [57]:
raw_tally = pd.read_csv(data_folder+'data/final_UK/adverse_events_all_labels_w_freq.csv')
raw_tally.head(1)

,product_id,freq,pt_meddra_term,pt_meddra_id,ingredient_rxnorm,active_ingredients
0,4223,not known,hypersensitivity,10020751,"22906, 236248","amylmetacresol, 2,4-dichlorobenzyl alcohol"


In [66]:
uniq_ingr = drug_df[['product_id', 'product_name', 'active_ingredients', 'ingredient_rxnorm']]
uniq_ingr['num_ingredients'] = uniq_ingr.ingredient_rxnorm.apply(lambda x: len(x.split(', ')) if str(x) != 'nan' else 0)
uniq_ingr = uniq_ingr.groupby('ingredient_rxnorm')['product_name'].apply(list).reset_index()
uniq_ingr['num_labels'] = uniq_ingr.product_name.apply(lambda x: len(x))
uniq_event = raw_tally[['pt_meddra_id', 'pt_meddra_term']].drop_duplicates()

In [69]:
import collections
ade_count = raw_tally[['ingredient_rxnorm', 'pt_meddra_id']].groupby('ingredient_rxnorm')['pt_meddra_id'].apply(list).reset_index()
ade_count['pt_meddra_list'] = ade_count['pt_meddra_id'].apply(lambda x: collections.Counter(x))
ade_count['pt_meddra_id'] = ade_count['pt_meddra_list'].apply(lambda x: x.keys())
ade_count = ade_count.explode('pt_meddra_id')
ade_count['num_events'] = ade_count.apply(lambda x: x['pt_meddra_list'][x['pt_meddra_id']], axis = 1)
ade_count.head(1)

,ingredient_rxnorm,pt_meddra_id,pt_meddra_list,num_events
0,1000112,10006187,"{10006187: 3, 10028997: 3, 10002034: 1, 100615...",3


In [71]:
ades = uniq_ingr.merge(ade_count[['ingredient_rxnorm', 'pt_meddra_id', 'num_events']], on = 'ingredient_rxnorm', how = 'left')
ades['pct_labels'] = ades.apply(lambda x: x['num_events'] / x['num_labels'], axis = 1)
ades = ades.merge(uniq_event, on = 'pt_meddra_id', how = 'left')
ades.head(1)

,ingredient_rxnorm,product_name,num_labels,pt_meddra_id,num_events,pct_labels,pt_meddra_term
0,1000112,[Depo-Provera 150mg/ml Injection Sterile suspe...,9,10006187,3.0,0.333333,breast cancer


In [80]:
a = raw_tally.drop('freq', axis = 1).drop_duplicates()
a = a.merge(ades[['ingredient_rxnorm', 'pt_meddra_id', 'pct_labels', 'num_labels', 'num_events']], on = ['ingredient_rxnorm', 'pt_meddra_id'], how = 'left')
a = a.drop_duplicates()
a.to_csv(data_folder+'data/final_UK/adverse_events.csv', index=False)
a.head(1)

,product_id,pt_meddra_term,pt_meddra_id,ingredient_rxnorm,active_ingredients,pct_labels,num_labels,num_events
0,4223,hypersensitivity,10020751,"22906, 236248","amylmetacresol, 2,4-dichlorobenzyl alcohol",1.0,5.0,5.0


----
## out of date code

In [67]:
onsides_df = pd.DataFrame()
for ingredient in tqdm(drug_df.ingredient_rxnorm.unique().tolist()):
  product_list = drug_df[drug_df.ingredient_rxnorm == ingredient].product_id.tolist()
  num_labels = len(product_list)
  filtered_conditions = conditions[conditions.product_id.isin([int(i) for i in product_list])].drop_duplicates()
  individual_df = filtered_conditions.pt_meddra_id.value_counts().reset_index()
  individual_df.columns = ['pt_meddra_id', 'percent_labels']
  individual_df['ingredients_rxcuis'] = ingredient
  individual_df['num_labels'] = num_labels
  individual_df['percent_labels'] = individual_df.percent_labels.apply(lambda x: x / num_labels)
  onsides_df = onsides_df.append(individual_df)
onsides_df.head()

100%|██████████| 1836/1836 [00:47<00:00, 38.81it/s]


,pt_meddra_id,percent_labels,ingredients_rxcuis,num_labels


In [40]:
onsides_df = onsides_df.merge(drug_df[['ingredient_rxnorm', 'active_ingredients']], left_on = 'ingredients_rxcuis', right_on = 'ingredient_rxnorm', how = 'left')\
.merge(conditions[['pt_meddra_id', 'pt_meddra_term']].drop_duplicates(), on = 'pt_meddra_id', how = 'left')
onsides_df.head()

,percent_labels,ingredients_rxcuis,num_labels,ingredient_rxnorm,active_ingredients,pt_meddra_id,pt_meddra_term


In [41]:
#onsides_df = onsides_df.drop('ingredient_rxnorm', axis = 1)
onsides_df = onsides_df[['ingredients_rxcuis', 'active_ingredients', 'pt_meddra_id', 'pt_meddra_term', 'percent_labels', 'num_labels']]
onsides_df.head(1)

,ingredients_rxcuis,active_ingredients,pt_meddra_id,pt_meddra_term,percent_labels,num_labels


In [42]:
onsides_df.to_csv(data_folder+'data/final_UK/adverse_events_wo_freq.csv', index=False)

In [ ]:
c = conditions
c.product_id = c.product_id.astype(str)
c = c.merge(drug_df[['ingredient_rxnorm', 'product_id']], on = 'product_id', how = 'left')[['ingredient_rxnorm', 'pt_meddra_id', 'freq']].drop_duplicates()\
.groupby(by=['ingredient_rxnorm', 'pt_meddra_id'])['freq'].apply(list).reset_index()
c.head(1)

,ingredient_rxnorm,pt_meddra_id,freq
0,,10000059,"[uncommon, not known, very common]"


In [ ]:
#add frequency
onsides_df = onsides_df.merge(c, left_on = ['ingredients_rxcuis', 'pt_meddra_id'], right_on = ['ingredient_rxnorm', 'pt_meddra_id'], how = 'left')
onsides_df = onsides_df.drop(['ingredient_rxnorm'],axis = 1)
onsides_df.head(1)

,ingredients_rxcuis,active_ingredients,pt_meddra_id,pt_meddra_term,percent_labels,num_labels,freq
0,"22906, 236248","amylmetacresol, 2,4-dichlorobenzyl alcohol",10020751,hypersensitivity,1.0,5,[not known]


In [ ]:
onsides_df.to_csv(data_folder+'onsides_table_w_freq.csv', index=False)

In [ ]:
print(onsides_df.shape)

(622235, 7)
